# **_004_API Data Extraction_**

In [ ]:
import pandas as pd
import requests
import time
import random
import html
from tqdm import tqdm


In this part we clean a list of artist names and retrieve additional data from Wikidata (like country, awards, gender, and number of albums).

- Step 1: Clean the artist names

    - Function limpiar_nombre(nombre):

    - Removes unwanted characters like backslashes, quotes, commas, slashes, and ampersands.

    - Trims spaces and returns a clean name or None if empty.

- Step 2: Load and preprocess the data

    - Read the CSV file artists.csv (without header).

    - Apply cleaning function to each name.

    - Remove duplicates and sort the list.

- Step 3: Prepare SPARQL query

    - Function construir_query_sparql(artistas):

    - Builds a SPARQL query dynamically for a list of artists.

    - Retrieves:

        - Artist label

        - Country

        - Award

        - Gender

        - Album count

    - Uses optional clauses to get data only if available.

    - Groups the results to avoid duplicates.

- Step 4: Query Wikidata API

    - Function obtener_datos_wikidata(artistas_batch):

    -  Sends the SPARQL query to Wikidata’s endpoint.

    - Handles HTTP errors gracefully.

- Step 5: Process artists in batches

    - To respect the query size limit:

    - Process artists in batches of up to 80.

    - Adjusts the batch size if the query exceeds the byte limit (60,000 bytes).

    - Uses tqdm for progress visualization.

- Step 6: Store the results

    - Collects the retrieved data into a list of dictionaries (results).

    - For each row returned, stores:

        - Artist name

        - Country

        - Award

        - Gender

        - Album count

In [ ]:
import pandas as pd
import re
import time
import requests
from tqdm import tqdm

def limpiar_nombre(nombre):
    if pd.isna(nombre) or not nombre.strip():
        return None
    nombre = nombre.replace("\\", "").replace('"', '').replace("'", "")
    nombre = nombre.replace(",", "").replace("/", " ").replace("&", "and")
    return nombre.strip()

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])
nombres_limpios = [limpiar_nombre(nombre) for nombre in df["raw"]]
artistas_unicos = sorted(set([nombre for nombre in nombres_limpios if nombre]))

print(f"Total artistas únicos: {len(artistas_unicos)}")

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
HEADERS = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "Workshop/1.0 (tucorreo@example.com)"
}

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?countryLabel ?awardLabel ?genderLabel (COUNT(?album) AS ?albumCount) WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      OPTIONAL {{ ?artist wdt:P166 ?award. }}
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}

      OPTIONAL {{
        ?album wdt:P31 wd:Q482994.
        ?album wdt:P175 ?artist.
      }}

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    GROUP BY ?artistLabel ?countryLabel ?awardLabel ?genderLabel
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Error en SPARQL:", e)
        return None

print("Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 80
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown"),
                        "album_count": row.get("albumCount", {}).get("value", "0")
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(0.8)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

columnas_ordenadas = ["artist", "country", "award", "gender", "album_count"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)


✅ Total artistas únicos: 31437
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 31440it [10:59, 47.69it/s]                           


Finally, the data is split in two and then the extracted data is saved in two different files.

In [ ]:
halfway = len(df_result) // 2

df_result.iloc[:halfway].to_csv("../data/api_data_part1.csv", index=False)
print("Archivo 'api_data_part1.csv' creado correctamente.")

df_result.iloc[halfway:].to_csv("../data/api_data_part2.csv", index=False)
print("Archivo 'api_data_part2.csv' creado correctamente.")

✅ Archivo 'api_data_part1.csv' creado correctamente.
✅ Archivo 'api_data_part2.csv' creado correctamente.
